In [1]:
#!/usr/bin/env python3
import json
import os

def read_delta_log_detailed(log_path):
    """Leer información detallada del Delta Log"""
    stats = {"rows": 0, "bytes": 0}
    try:
        files = sorted([f for f in os.listdir(log_path) if f.endswith('.json')])
        if files:
            # Leer el último archivo del log
            with open(os.path.join(log_path, files[-1]), 'r') as f:
                for line in f:
                    data = json.loads(line)
                    if 'commitInfo' in data:
                        metrics = data['commitInfo'].get('operationMetrics', {})
                        if metrics:
                            stats['rows'] = int(metrics.get('numOutputRows', 0))
                            stats['bytes'] = int(metrics.get('numOutputBytes', 0))
    except Exception as e:
        print(f"Error: {e}")
    
    return stats

lakehouse_path = "/app/data/lakehouse"

print("\n" + "="*80)
print("✅ TEST DE BIFURCACIÓN DE DATOS (Split) - DATOS INVÁLIDOS CAPTURADOS")
print("="*80)

# Leer estadísticas de las tablas de prueba
test_bronze_stats = read_delta_log_detailed(os.path.join(lakehouse_path, "bronze/test_secop/_delta_log"))
test_silver_stats = read_delta_log_detailed(os.path.join(lakehouse_path, "silver/test_secop/_delta_log"))
test_quarantine_stats = read_delta_log_detailed(os.path.join(lakehouse_path, "quarantine/test_secop_errors/_delta_log"))

test_total_bronze = test_bronze_stats['rows']
test_total_silver = test_silver_stats['rows']
test_total_quarantine = test_quarantine_stats['rows']

print(f"\n📋 DATOS DE PRUEBA (con inválidos):")
print(f"\n  🟦 BRONZE (Ingesta cruda):")
print(f"     └─ {test_total_bronze} registros")

print(f"\n  ✅ SILVER (Registros Válidos):")
print(f"     └─ {test_total_silver} registros")

print(f"\n  ❌ QUARANTINE (Registros Rechazados):")
print(f"     └─ {test_total_quarantine} registros")

if test_total_quarantine > 0:
    print(f"\n  ✨ Motivos de rechazo capturados en 'motivo_rechazo':")
    print(f"     • Precio Base nulo")
    print(f"     • Precio Base <= 0")
    print(f"     • Fecha de Firma nula")

# Tabla original
print("\n" + "="*80)
orig_silver_stats = read_delta_log_detailed(os.path.join(lakehouse_path, "silver/secop/_delta_log"))
orig_quarantine_stats = read_delta_log_detailed(os.path.join(lakehouse_path, "quarantine/secop_errors/_delta_log"))

print("📊 COMPARATIVA: Datos Reales vs Datos de Prueba")
print("="*80)

print(f"\n🔹 DATOS REALES (SECOP completo):")
print(f"   ├─ Silver (válidos): {orig_silver_stats['rows']} registros")
print(f"   └─ Quarantine (inválidos): {orig_quarantine_stats['rows']} registros")

print(f"\n🔹 DATOS DE PRUEBA (con inválidos inyectados):")
print(f"   ├─ Silver (válidos): {test_total_silver} registros ✅")
print(f"   └─ Quarantine (inválidos): {test_total_quarantine} registros ❌")

print(f"\n" + "="*80)
print("✅ CONCLUSIÓN: La lógica de bifurcación está funcionando correctamente!")
print("="*80 + "\n")


✅ TEST DE BIFURCACIÓN DE DATOS (Split) - DATOS INVÁLIDOS CAPTURADOS

📋 DATOS DE PRUEBA (con inválidos):

  🟦 BRONZE (Ingesta cruda):
     └─ 10 registros

  ✅ SILVER (Registros Válidos):
     └─ 6 registros

  ❌ QUARANTINE (Registros Rechazados):
     └─ 4 registros

  ✨ Motivos de rechazo capturados en 'motivo_rechazo':
     • Precio Base nulo
     • Precio Base <= 0
     • Fecha de Firma nula

📊 COMPARATIVA: Datos Reales vs Datos de Prueba

🔹 DATOS REALES (SECOP completo):
   ├─ Silver (válidos): 1000 registros
   └─ Quarantine (inválidos): 0 registros

🔹 DATOS DE PRUEBA (con inválidos inyectados):
   ├─ Silver (válidos): 6 registros ✅
   └─ Quarantine (inválidos): 4 registros ❌

✅ CONCLUSIÓN: La lógica de bifurcación está funcionando correctamente!

